In [1]:
import pandas as pd 

In [2]:
wedding = pd.read_csv('wedding_merged.csv')
cheese = pd.read_csv('cheese_merged.csv')

In [ ]:
# Nastavení primárních klíčů
cheese['cheeses_ID'] = cheese['cheese']  # Použití názvu sýra jako primárního klíče
wedding['wedding_ID'] = wedding['Type of Wine'] + " - " + wedding['Wine Grape']  # Kombinace sloupců Type of Wine a Wine Grape  

In [4]:
# Cross join tabulek (každý řádek s každým řádkem)
cross_join = cheese.merge(wedding, how='cross')

In [5]:
# Výběr sloupců, které mají True/False hodnoty
boolean_columns = [col for col in cheese.columns if col in wedding.columns and cheese[col].dtype == 'bool' and col[:7] != 'family_']
num_boolean_columns = len(boolean_columns) +3  # Počet sloupců pro výpočet procenta
num_boolean_columns

35

In [6]:
# # Výpočet skóre

# cross_join['score'] = 0
# for col in boolean_columns:
#     cross_join['score'] += (cross_join[col + '_x'] == cross_join[col + '_y']).astype(int)
# if cross_join['family_new_x'] == "gouda": #cross_join['family_new_y']:
#     cross_join['score'] += 3 

# Výpočet skóre
if not boolean_columns:
    print("Žádné boolean sloupce pro porovnání!")
else:
    cross_join['score'] = 0
    for col in boolean_columns:
        cross_join['score'] += (cross_join[col + '_x'] == cross_join[col + '_y']).astype(int)

    # Zvýšení skóre o 3, pokud se family_new_x a family_new_y shodují
    cross_join['score'] += (cross_join['family_new_x'] == cross_join['family_new_y']).astype(int) * 3


In [ ]:
# Výpočet procentuální shody
cross_join['match_percentage'] = round((cross_join['score'] / num_boolean_columns), 2).astype(float)


In [8]:
# Přeskupení a agregace dat
score_tab = cross_join.groupby(['cheeses_ID', 'wedding_ID'], as_index=False).agg(
    MAX_SCORE=('score', 'max'),
    MAX_MATCH_PERCENTAGE=('match_percentage', 'max')
)



In [9]:
score_tab.to_csv('score_tab.csv', index=False)


In [10]:
score_tab.head()

,cheeses_ID,wedding_ID,MAX_SCORE,MAX_MATCH_PERCENTAGE
0,ADL Brick Cheese,port - Gouveio,21,0.60
1,ADL Brick Cheese,port - Malvasia Fina,21,0.60
2,ADL Brick Cheese,port - Sousão,18,0.51
3,ADL Brick Cheese,port - Tempranillo,27,0.77
4,ADL Brick Cheese,port - Tinta Amarela,27,0.77
